In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score, make_scorer
import pickle

In [2]:
X = pd.read_csv(r'C:\Users\mj\Desktop\ML-\sales_train.csv')
X

,date,item_id,price,item_count
0,2019-01-01,3,29.22,2.0
1,2019-01-01,4,26.42,22.0
2,2019-01-01,12,4.87,7.0
3,2019-01-01,13,4.18,12.0
4,2019-01-01,16,3.21,136.0
...,...,...,...,...
109595,2021-12-31,96,21.93,0.0
109596,2021-12-31,97,28.65,0.0
109597,2021-12-31,98,5.00,0.0
109598,2021-12-31,99,5.32,0.0


In [3]:
# i realized that i can aggregate my predition instade 


In [4]:
X_test_full = pd.read_csv(r'C:\Users\mj\Desktop\test_data.csv')
X_test_full

,date,item_id,price,item_count
0,2021-10-26,67,7.00,5.0
1,2019-01-08,67,7.00,9.0
2,2019-01-23,12,4.87,5.0
3,2020-10-08,15,18.78,0.0
4,2020-11-03,50,26.37,0.0
...,...,...,...,...
21915,2021-08-27,76,17.93,97.0
21916,2021-08-27,62,5.70,0.0
21917,2020-09-30,20,7.95,0.0
21918,2019-08-28,76,17.93,54.0


In [5]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['price'], inplace=True)
y = X.price              
X.drop(['price'], axis=1, inplace=True)


In [6]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)



In [7]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [8]:
my_model_1 = XGBRegressor(random_state=0)
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=0, ...)

In [9]:
predictions_1 = my_model_1.predict(X_valid)

In [10]:
mae_1 = mean_absolute_error(predictions_1, y_valid)
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 0.012082826777096193


In [11]:
predictions_1

array([ 6.9963913,  6.918841 ,  4.8617983, ...,  7.9610505, 17.930777 ,
       15.694328 ], dtype=float32)

In [12]:
# Combine predicted and actual values into a DataFrame
predictions_df = pd.DataFrame({'Actual': y_valid, 'Predicted': predictions_1})
#j = predictions_df.to_json(orient='records')

# Print the DataFrame
print(predictions_df)

        Actual  Predicted
102915    7.00   6.996391
712       7.00   6.918841
2202      4.87   4.861798
64633    18.78  18.767939
67257    26.37  26.351900
...        ...        ...
96921    17.93  17.944063
96968     5.70   5.700958
63837     7.95   7.961051
23917    17.93  17.930777
81966    15.69  15.694328

[21920 rows x 2 columns]


In [13]:
# Aggregate predictions and actual values based on item ID
aggregated_predictions = predictions_df.groupby(X_valid['item_id']).mean()

# Print the aggregated predictions
print(aggregated_predictions)

         Actual  Predicted
item_id                   
1          6.71   6.713000
2         15.09  15.090527
3         29.22  29.211473
4         26.42  26.420633
5          6.07   6.071676
...         ...        ...
96        21.93  21.926071
97        28.65  28.641819
98         5.00   5.001775
99         5.32   5.321807
100        2.48   2.482734

[100 rows x 2 columns]


In [14]:
item_names_df = pd.read_csv(r'C:\Users\mj\Desktop\items.csv')
# Rename the 'id' column to 'item_id'
item_names_df.rename(columns={'id': 'item_id'}, inplace=True)

# Drop the 'store_id', 'kcal', and 'cost' columns
item_names_df.drop(['store_id', 'kcal', 'cost'], axis=1, inplace=True)


# Merge the predictions DataFrame with item_names_df based on item_id
predictions_with_names = pd.merge(aggregated_predictions, item_names_df, on='item_id')



In [15]:
# Print the updated DataFrame
predictions_with_names.head(10)

,item_id,Actual,Predicted,name
0,1,6.71,6.713000,Chocolate Cake
1,2,15.09,15.090527,Breaded Fish with Vegetables Meal
2,3,29.22,29.211473,Sweet Fruity Cake
3,4,26.42,26.420633,Amazing Steak Dinner with Rolls
4,5,6.07,6.071676,Milk Cake
5,6,5.61,5.616941,Fruity Frozen Cocktail
6,7,8.10,8.102676,Fruity Milky Cake
7,8,24.98,24.970034,Amazing Trout with Vegetables Dinner
8,9,3.91,3.915596,Orange Juice
9,10,21.13,21.123741,Mutton with Roles and Vegetables Plate


In [16]:
# Define the pipeline
pipeline = Pipeline(steps=[("model", my_model_1)])

# Perform cross-validation
scores = -cross_val_score(
    pipeline, X_train, y_train, cv=5, scoring="neg_mean_absolute_error"
)

# Print the mean absolute error for each fold
print("Mean Absolute Errors:", scores)


Mean Absolute Errors: [0.01908327 0.01484061 0.01520228 0.01641554 0.01624918]


In [17]:
scorer = make_scorer(r2_score)


# Perform cross-validation
scores = cross_val_score(
    pipeline, X_train, y_train, cv=5, scoring=scorer
)

# Print the mean absolute error for each fold
print("R2 :", scores)


R2 : [0.99994512 0.99999216 0.9999875  0.99998294 0.99997942]


In [18]:
scorer2 = make_scorer(r2_score)


# Perform cross-validation
scores2 = cross_val_score(
    pipeline, X_valid, y_valid, cv=5, scoring=scorer2
)

# Print the mean absolute error for each fold
print("R2 :", scores2)


R2 : [0.99996177 0.99996615 0.99987725 0.99984563 0.99837062]


In [19]:
#saving the model 
filename = "pricing_model.pkl"
pickle.dump(my_model_1, open(filename, "wb"))

In [20]:
predictions_2 = my_model_1.predict(X_train)

In [21]:
predictions_2

array([18.511995 , 15.226254 , 15.694328 , ..., 13.6200905,  3.7549734,
       15.694328 ], dtype=float32)